In [3]:
import large_image
import os
#! pip install git+https://github.com/DigitalSlideArchive/DSA-WSI-DeID.git
sampleSVSfile = '/Users/dagutman/devel/nci-dsa-deid/devops/nci-dsa-deid/import/SmallSampleFiles/TCGA-06-5416-01A-01-TS1.1749a247-c33d-4a06-88f2-4deef3c5d982.svs'


from girder_large_image.models import image_item


class DeIdImageItem:
    def __init__(self, filePath=None, metadata=None):
        # print(filePath,"is the file path")
        self.metadata = metadata if metadata else {}
        self.filePath = filePath
        self.filename = os.path.basename(filePath) if self.filePath else None
        
      #  self.largeImage = self.tileSource.getMetadata() if self.tileSource else None
        self.meta = {'largeImage': True}
        self._largeImagePath = self.filePath

    def update_metadata(self, new_metadata):
        self.metadata.update(new_metadata)

    def get_metadata(self):
        return self.metadata
    
    def get_tile_source(self, *args, **kwargs):
        # Your implementation here
        print("Crap nuggets")
        pass
    
    def __iter__(self):
        for key, value in self.meta.items():
            yield (key, value)

    def tileSource(self,  item, **kwargs):
        ts = large_image.open(item.filePath)
        #self.tileSource = large_image.open(self.filePath) if self.filePath else None
        return ts
    
    def getMetadata(self, item, **kwargs):
        
        return self.tileSource().getMetadata()
    
    def get(self, key, default=None):
        return self.meta.get(key, default)

    def _largeImagePath(self, item, **kwargs):
        return self.filePath

image_item.ImageItem = DeIdImageItem

ImageItem = DeIdImageItem
# from girder_large_image.models.image_item import ImageItem




In [4]:
sampleSlide = DeIdImageItem(filePath=sampleSVSfile)
from wsi_deid.process import get_standard_redactions
redacts = get_standard_redactions(sampleSlide, "ToBeRenamed.svs")

# wsi_deid.process.get_standard_redactions(sampleSlide, "ToBeRenamed.svs")
  

In [3]:
import tifftools
# ts = ImageItem().tileSource(sampleSlide)
# sourcePath = ts._getLargeImagePath()
# tiffinfo = tifftools.read_tiff(sourcePath)

from large_image_source_tiff import TiffFileTileSource
sampleSVSfile = '/Users/dagutman/devel/nci-dsa-deid/devops/nci-dsa-deid/import/SmallSampleFiles/TCGA-06-5416-01A-01-TS1.1749a247-c33d-4a06-88f2-4deef3c5d982.svs'

import large_image_source_tiff.girder_source    
# large_image_source_tiff.girder_source.TiffGirderTileSource = TiffFileTileSource
# tiffSource = large_image_source_tiff.girder_source.TiffGirderTileSource(sampleSVSfile)
##tiffSource = large_image_source_tiff.girder_source.TiffGirderTileSource(item)

tiffSource = large_image_source_tiff.TiffFileTileSource(sampleSVSfile)

: 

In [ ]:

def redact_format_aperio(item, tempdir, redactList, title, labelImage, macroImage):
    """
    Redact aperio files.

    :param item: the item to redact.
    :param tempdir: a directory for work files and the final result.
    :param redactList: the list of redactions (see get_redact_list).
    :param title: the new title for the item.
    :param labelImage: a PIL image with a new label image.
    :param macroImage: a PIL image with a new macro image.  None to keep or
        redact the current macro image.
    :returns: (filepath, mimetype) The redacted filepath in the tempdir and
        its mimetype.
    """
    import large_image_source_tiff.girder_source

    tileSource = ImageItem().tileSource(item)
    sourcePath = tileSource._getLargeImagePath()

    tiffinfo = tifftools.read_tiff(sourcePath)
    ifds = tiffinfo["ifds"]
    if redactList.get("area", {}).get("_wsi", {}).get("geojson"):
        ifds = redact_format_aperio_philips_redact_wsi(
            tileSource, ifds, redactList["area"]["_wsi"]["geojson"], tempdir
        )
        ImageItem().removeThumbnailFiles(item)
    aperioValues = aperio_value_list(item, redactList, title)
    imageDescription = "|".join(aperioValues)
    # We expect aperio to have the full resolution image in directory 0, the
    # thumbnail in directory 1, lower resolutions starting in 2, and label and
    # macro images in other directories.  Confirm this -- our tiff reader will
    # report the directories used for the full resolution.
    tiffSource = large_image_source_tiff.girder_source.TiffGirderTileSource(
        sampleSVSfile
    )
    mainImageDir = [
        dir._directoryNum for dir in tiffSource._tiffDirectories[::-1] if dir
    ]
    associatedImages = tileSource.getAssociatedImagesList()
    if mainImageDir != [
        d + (1 if d and "thumbnail" in associatedImages else 0)
        for d in range(len(mainImageDir))
    ]:
        raise Exception("Aperio TIFF directories are not in the expected order.")
    firstAssociatedIdx = max(mainImageDir) + 1
    # Set new image description
    ifds[0]["tags"][tifftools.Tag.ImageDescription.value] = {
        "datatype": tifftools.Datatype.ASCII,
        "data": imageDescription,
    }
    # redact or adjust thumbnail
    if "thumbnail" in associatedImages:
        if "thumbnail" in redactList["images"]:
            ifds.pop(1)
            firstAssociatedIdx -= 1
        else:
            thumbnailComment = ifds[1]["tags"][tifftools.Tag.ImageDescription.value][
                "data"
            ]
            thumbnailDescription = "|".join(
                thumbnailComment.split("|", 1)[0:1] + aperioValues[1:]
            )
            ifds[1]["tags"][tifftools.Tag.ImageDescription.value][
                "data"
            ] = thumbnailDescription
    # redact other images
    for idx in range(len(ifds) - 1, 0, -1):
        ifd = ifds[idx]
        key = None
        keyparts = (
            ifd["tags"]
            .get(tifftools.Tag.ImageDescription.value, {})
            .get("data", "")
            .split("\n", 1)[-1]
            .strip()
            .split()
        )
        if len(keyparts) and keyparts[0].lower() and not keyparts[0][0].isdigit():
            key = keyparts[0].lower()
        if (
            key is None
            and ifd["tags"].get(tifftools.Tag.NewSubfileType.value)
            and ifd["tags"][tifftools.Tag.NewSubfileType.value]["data"][0]
            & tifftools.Tag.NewSubfileType.bitfield.ReducedImage.value
        ):
            key = (
                "label"
                if ifd["tags"][tifftools.Tag.NewSubfileType.value]["data"][0] == 1
                else "macro"
            )
        if (
            key in redactList["images"]
            or key == "label"
            or (key == "macro" and macroImage)
        ):
            ifds.pop(idx)
    # Add back label and macro image
    if macroImage:
        redact_format_aperio_add_image(
            "macro", macroImage, ifds, firstAssociatedIdx, tempdir, aperioValues
        )
    if labelImage:
        redact_format_aperio_add_image(
            "label", labelImage, ifds, firstAssociatedIdx, tempdir, aperioValues
        )
    # redact general tiff tags
    redact_tiff_tags(ifds, redactList, title)
    add_deid_metadata(item, ifds)
    outputPath = os.path.join(tempdir, "aperio.svs")
    tifftools.write_tiff(ifds, outputPath)
    # logger.info("Redacted aperio file %s as %s", sourcePath, outputPath)
    return outputPath, "image/tiff"

In [ ]:
  # def get_dict(self):
    #     return self.meta

    # @classmethod
    # def _loadTileSource(cls, item, **kwargs):
    #     if 'largeImage' not in item:
    #         print("Tile Source not found")  
    #     else:
    #         ts = large_image.open(item.filePath)
    #         return ts    


# class DeIdImageItem():
#     def __init__(self, filePath=None, metadata=None):
#         self.metadata = metadata if metadata else {}
#         self.filename = os.path.basename(filePath)
#         self.filePath = filePath
#         self.tileSource = large_image.open(filePath)
#         self.largeImage = self.tileSource.getMetadata()
#         self.meta = {'largeImage': True}
#     def update_metadata(self, new_metadata):
#         self.metadata.update(new_metadata)

#     def get_metadata(self):
#         return self.metadata
    
#     def tileSource(self, *args, **kwargs):
#         # Your implementation here
#         print("Crap nuggets")
#         pass
    
#     def __iter__(self):
#         for key, value in self.meta.items():
#             yield (key, value)